In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import seaborn as sns
import seaborn as sb
from config import gkey

gmaps.configure(api_key=gkey)

In [2]:
construction_loss = pd.read_csv ('Fire.csv')
construction_loss.head()

,index,ID,DATE,TIME,ZIP,BATTALION,SHIFT,COUNCIL DISTRICT,STRUCTURE TYPE,BUILDING STORIES,...,TOTAL VALUE,STRUCTURE LOSS,CONTENTS LOSS,TOTAL SAVED,PERCENTAGE LOSS,SPRINKER SYSTEM,ALARM SYSTEM,ADDRESS,Lat,Lng
0,0,17961.0,10/29/2019,08:21 PM,75216.0,6,A,4.0,DWELLING,1.0,...,166370.0,-5000.0,0.0,161370.0,3.01,NaN,NaN,1325 S MARSALIS AVE,32.732107,-96.814783
1,1,17962.0,10/30/2019,03:42 AM,75229.0,7,A,13.0,DWELLING,1.0,...,535470.0,-10000.0,0.0,525470.0,1.87,NaN,NaN,10728 LATHROP DR,32.894248,-96.857108
2,2,17963.0,10/30/2019,10:47 AM,75241.0,5,B,3.0,DWELLING,1.0,...,306000.0,-5000.0,-15000.0,286000.0,6.54,NaN,NaN,6205 S R L THORNTON SERV,32.694543,-96.824020
3,3,17964.0,10/30/2019,05:32 PM,75243.0,2,B,10.0,DWELLING,1.0,...,126710.0,-8000.0,-4000.0,114710.0,9.47,NaN,NaN,12044 WILLOWDELL DR,32.913584,-96.764435
4,4,17965.0,10/30/2019,11:03 AM,75212.0,9,B,6.0,DWELLING,1.0,...,84000.0,-45000.0,-10000.0,29000.0,65.48,NaN,NaN,2212 LAPSLEY ST,32.769986,-96.908816


In [6]:
construction_loss["CONSTRUCTION"].count()

542

In [7]:
construction_loss["CONSTRUCTION"].value_counts

<bound method IndexOpsMixin.value_counts of 0       WOOD
1       WOOD
2       WOOD
3       WOOD
4       WOOD
       ...  
616     WOOD
617     WOOD
618     WOOD
619    METAL
620     WOOD
Name: CONSTRUCTION, Length: 621, dtype: object>

In [8]:
#unique structures included in the data set
unique_construction_df = construction_loss["CONSTRUCTION"].unique()

unique_construction_df

array(['WOOD', 'BRICK', nan, 'MASONRY', 'METAL', 'FRAME', 'BLOCK',
       'CONCRETE', 'STUCCO', 'COMBO', 'STEEL'], dtype=object)

In [10]:
#average percent loss by structure type
avg_loss_construction = construction_loss.groupby(["CONSTRUCTION"]).mean()["PERCENTAGE LOSS"]

avg_loss_construction

CONSTRUCTION
BLOCK       13.000000
BRICK        8.639459
COMBO        0.050000
CONCRETE     0.084167
FRAME       24.490000
MASONRY      3.975000
METAL       22.154444
STEEL        3.587500
STUCCO       0.700000
WOOD        11.623185
Name: PERCENTAGE LOSS, dtype: float64

In [ ]:
#what does frame stand for?
#BLOCK=Concrete/Cinder Block

#Top 5 
# 1. Steel
# 2. Concrete 
# 3. Wood
# 4. Stone
# 5. Brick/Masonry

#CONCRETE     0.084167
#FRAME(24.490000) and WOOD(11.623185)
#MASONRY(3.975000) and BRICK(8.639459) and BLOCK(13.000000)
#STEEL(3.587500) and METAL(22.154444)

#COMBO        0.050000
#STUCCO       0.700000
